In [1]:
import numpy as np
import pandas as pd

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

def reduce_mem_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
 
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [2]:
data = reduce_mem_usage(pd.read_csv('final2_prosper_dataset.csv'))
data['average_income'] = (data['MinIncome']+ data['MaxIncome'])/2
data.drop('Unnamed: 0',axis=1,inplace=True)
data.drop(["ListingCreationDate","FirstRecordedCreditLine","IncomeRange","LoanOriginationDate","DateCreditPulled"],axis=1,inplace=True)
risk = ['ProsperRating (numeric)','ProsperRating (Alpha)','ProsperScore','high_risk']
CreditScore = ['CreditScoreRangeLower','CreditScoreRangeUpper']
df = data.copy()
df.drop(CreditScore,axis=1,inplace=True)
df = df.drop(risk,axis=1).drop('LoanKey',axis=1)
df = df.drop(['MinIncome','MaxIncome'],axis=1)
df.info()

Memory usage of dataframe is 81.09 MB --> 33.53 MB (Decreased by 58.6%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113066 entries, 0 to 113065
Data columns (total 80 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   CreditGrade                         113066 non-null  object 
 1   Term                                113066 non-null  int8   
 2   BorrowerAPR                         113066 non-null  float32
 3   BorrowerRate                        113066 non-null  float32
 4   LenderYield                         113066 non-null  float32
 5   EstimatedEffectiveYield             113066 non-null  float32
 6   EstimatedLoss                       113066 non-null  float32
 7   EstimatedReturn                     113066 non-null  float32
 8   ListingCategory (numeric)           113066 non-null  int8   
 9   BorrowerState                       113066 non-null  object 
 10  Occupation          

In [3]:
X = pd.get_dummies(df,columns=[col for col in df.columns if 'O' == df[col].dtype])

Data = data.copy()
# Transform into a categorical variable
Data['high_risk'] = pd.Categorical(Data['high_risk'])

# Assign a number to each category (label encoding)
Data['high_risk'] = Data['high_risk'].cat.codes 

y = Data.drop(['ProsperRating (Alpha)'], axis=1)
# Use to_categorical on your labels
y = to_categorical(Data['high_risk'])

# Now print the one-hot encoded labels
print('One-hot encoded high risk: \n',y)
print(y.shape)

One-hot encoded high risk: 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
(113066, 2)


In [4]:
predictors, test_X, target, test_y = train_test_split(X, y, train_size = 0.75, random_state = 1111)
print(predictors.shape)
print(test_X.shape)
print(target.shape)
print(test_y.shape)

(84799, 211)
(28267, 211)
(84799, 2)
(28267, 2)


In [14]:
# Instantiate a sequential model
model3 = Sequential()
  
# Add dense layers
model3.add(Dense(256, input_shape=(211,), activation='relu'))
model3.add(Dense(512, activation='relu'))
model3.add(Dense(256, activation='relu'))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(64, activation='relu'))
  
# Add a dense layer with as many neurons as prosper rating
model3.add(Dense(2, activation='softmax'))
  
# Compile your model using categorical_crossentropy loss
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
# Fit your model to the training data for 200 epochs
model3.fit(predictors, target, validation_split=0.25, epochs=50)

# Evaluate your model accuracy on the test data
accuracy = model3.evaluate(test_X, test_y)[1]

# Print accuracy
print('Accuracy:', accuracy)

# Predict on coords_small_test
preds = model3.predict(test_X)

# Print preds vs true values
print("{:45} | {}".format('Raw Model Predictions','True labels'))
for i,pred in enumerate(preds):
    print("{} | {}".format(pred,test_y[i]))

# Extract the position of highest probability from each pred vector
preds_chosen = [np.argmax(pred) for pred in preds]

# Print preds vs true values
print("{:10} | {}".format('Rounded Model Predictions','True labels'))
for i,pred in enumerate(preds_chosen):
    print("{:25} | {}".format(pred,test_y[i]))

Epoch 1/50
1988/1988 [==============================] - 9s 4ms/step - loss: 0.6302 - accuracy: 0.7019 - val_loss: 0.5615 - val_accuracy: 0.7168
Epoch 2/50
1988/1988 [==============================] - 8s 4ms/step - loss: 0.5774 - accuracy: 0.7166 - val_loss: 0.4657 - val_accuracy: 0.7624
Epoch 3/50
1988/1988 [==============================] - 7s 3ms/step - loss: 0.5272 - accuracy: 0.7367 - val_loss: 0.4877 - val_accuracy: 0.7567
Epoch 4/50
1988/1988 [==============================] - 7s 3ms/step - loss: 0.4799 - accuracy: 0.7539 - val_loss: 0.5280 - val_accuracy: 0.6827
Epoch 5/50
1988/1988 [==============================] - 7s 3ms/step - loss: 0.4616 - accuracy: 0.7643 - val_loss: 0.4450 - val_accuracy: 0.7849
Epoch 6/50
1988/1988 [==============================] - 7s 4ms/step - loss: 0.4395 - accuracy: 0.7806 - val_loss: 0.4355 - val_accuracy: 0.7742
Epoch 7/50
1988/1988 [==============================] - 7s 4ms/step - loss: 0.4368 - accuracy: 0.7834 - val_loss: 0.4631 - val_accuracy:

[0.01253404 0.9874659 ] | [0. 1.]
[0.33034703 0.6696529 ] | [0. 1.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.01253404 0.9874659 ] | [1. 0.]
[9.9999976e-01 1.8749571e-07] | [1. 0.]
[0.1396363 0.8603637] | [1. 0.]
[0.10417289 0.89582705] | [0. 1.]
[0.99853134 0.00146868] | [1. 0.]
[9.999912e-01 8.767056e-06] | [1. 0.]
[9.9946994e-01 5.3013960e-04] | [1. 0.]
[0.5306494  0.46935055] | [0. 1.]
[0.2873918  0.71260816] | [0. 1.]
[0.5887143  0.41128564] | [1. 0.]
[0.24377514 0.7562249 ] | [1. 0.]
[9.9912661e-01 8.7335013e-04] | [1. 0.]
[0.03556066 0.9644394 ] | [0. 1.]
[0.2996786 0.7003214] | [0. 1.]
[0.9171854 0.0828146] | [1. 0.]
[0.79114026 0.20885973] | [1. 0.]
[0.6555688  0.34443128] | [0. 1.]
[1.0000000e+00 3.1042205e-08] | [1. 0.]
[0.20387736 0.7961226 ] | [0. 1.]
[0.90628064 0.09371939] | [1. 0.]
[0.43170527 0.5682947 ] | [0. 1.]
[0.07160331 0.9283967 ] | [0. 1.]
[0.984832   0.01516801] | [1. 0.]
[0.59699875 0.40300125] | [1. 0.]
[0.47324055 0.5267595 ] | [1. 0.]
[0.3085778 0.6914222] | [0

[0.86034197 0.139658  ] | [0. 1.]
[0.17187282 0.8281271 ] | [1. 0.]
[9.999993e-01 6.880560e-07] | [1. 0.]
[9.991027e-01 8.973476e-04] | [1. 0.]
[0.10191416 0.8980859 ] | [0. 1.]
[1.0000000e+00 2.8027162e-17] | [1. 0.]
[9.9981862e-01 1.8136171e-04] | [1. 0.]
[0.7987742 0.2012258] | [0. 1.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.99790984 0.0020901 ] | [1. 0.]
[0.77511746 0.22488251] | [0. 1.]
[1.0000000e+00 3.8536022e-08] | [1. 0.]
[0.03497627 0.96502376] | [0. 1.]
[0.01253404 0.9874659 ] | [0. 1.]
[1.000000e+00 6.839234e-12] | [1. 0.]
[1.0000000e+00 3.6099226e-10] | [1. 0.]
[0.9861168  0.01388311] | [1. 0.]
[0.9673909  0.03260916] | [0. 1.]
[0.996549   0.00345098] | [1. 0.]
[0.04311873 0.9568813 ] | [0. 1.]
[0.8481974 0.1518026] | [1. 0.]
[0.92415637 0.07584364] | [1. 0.]
[0.6177381  0.38226193] | [0. 1.]
[0.40555707 0.5944429 ] | [0. 1.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.9825717 0.0174283] | [1. 0.]
[0.4162784  0.58372164] | [0. 1.]
[9.999950e-01 4.958519e-06] | [1. 0.]
[0.1322123 0.8

[9.9999988e-01 1.0970772e-07] | [1. 0.]
[0.6549987 0.3450013] | [1. 0.]
[0.28655252 0.71344745] | [0. 1.]
[9.9999952e-01 5.0925604e-07] | [1. 0.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.7382672  0.26173288] | [1. 0.]
[0.7558542  0.24414583] | [1. 0.]
[0.3547329 0.6452671] | [1. 0.]
[9.9999547e-01 4.5873830e-06] | [1. 0.]
[0.9740554  0.02594457] | [1. 0.]
[0.99715734 0.00284271] | [1. 0.]
[9.999999e-01 7.332699e-08] | [1. 0.]
[9.999304e-01 6.957058e-05] | [1. 0.]
[0.669305   0.33069497] | [1. 0.]
[0.99779487 0.00220512] | [1. 0.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.93919206 0.06080789] | [1. 0.]
[0.49526468 0.5047353 ] | [0. 1.]
[0.48446324 0.5155368 ] | [0. 1.]
[0.99387497 0.00612498] | [1. 0.]
[0.30534747 0.69465256] | [0. 1.]
[1.0000000e+00 2.5554585e-09] | [1. 0.]
[0.99249035 0.0075096 ] | [1. 0.]
[1.0000000e+00 1.5680068e-13] | [1. 0.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.88401353 0.11598642] | [1. 0.]
[0.88825583 0.11174424] | [1. 0.]
[9.999912e-01 8.785905e-06] | [1. 0.]
[0.2408964

[0.32566935 0.67433065] | [0. 1.]
[0.9988386  0.00116145] | [1. 0.]
[0.21344264 0.7865573 ] | [0. 1.]
[0.9275609  0.07243903] | [1. 0.]
[0.01253404 0.9874659 ] | [0. 1.]
[9.9999845e-01 1.5395897e-06] | [1. 0.]
[0.29793856 0.7020615 ] | [0. 1.]
[0.47424343 0.5257566 ] | [0. 1.]
[0.03970082 0.9602992 ] | [0. 1.]
[0.793972   0.20602801] | [0. 1.]
[0.28409696 0.71590304] | [0. 1.]
[0.3654519 0.6345481] | [0. 1.]
[1.000000e+00 4.386305e-11] | [1. 0.]
[9.99999881e-01 1.01853175e-07] | [1. 0.]
[0.99362224 0.00637773] | [1. 0.]
[0.5985545  0.40144548] | [1. 0.]
[0.18058313 0.8194169 ] | [0. 1.]
[0.9285718  0.07142818] | [1. 0.]
[0.01253404 0.9874659 ] | [0. 1.]
[0.9982224  0.00177757] | [1. 0.]
[0.2203082 0.7796919] | [0. 1.]
[0.96599823 0.03400179] | [1. 0.]
[0.8394541  0.16054586] | [1. 0.]
[1.0000000e+00 4.4379267e-10] | [1. 0.]
[0.30415055 0.6958494 ] | [1. 0.]
[9.9998236e-01 1.7699011e-05] | [1. 0.]
[0.25352776 0.74647224] | [0. 1.]
[0.16488542 0.83511454] | [0. 1.]
[0.73727    0.26272994

...

                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        1 | [0. 1.]
                        0 | [0. 1.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | 

                        0 | [1. 0.]
                        1 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | 

                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        1 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [0. 1.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | 

                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        1 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [0. 1.]
                        0 | 

                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        0 | [1. 0.]
                        0 | [1. 0.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        1 | [0. 1.]
                        0 | 

In [18]:
import pickle
with open("second_model3.pichle","wb") as file:
    pickle.dump(model1,file)
        
accuracy

INFO:tensorflow:Assets written to: ram://70277dda-5f80-4a72-9e2d-4114df58af44/assets


0.8404853940010071